# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
city_weather_df = pd.read_csv('cities_weather_df.csv')
city_weather_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=gkey)

cities = city_weather_df[['latitude','longitude']]
humidity = city_weather_df['humidity']

In [ ]:
fig = gmaps.figure(zoom_level=2, center=(0,0))
heat_layer = gmaps.heatmap_layer(cities, weights=humidity, 
                                 dissipating=False, max_intensity=130, point_radius=7, opacity=0.4)
fig.add_layer(heat_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
ideal_cities_df = city_weather_df.loc[(city_weather_df['temperature'] > 60) &
                                        (city_weather_df['temperature'] < 100) &
                                        (city_weather_df['humidity'] < 60) &
                                        (city_weather_df['cloudiness'] < 1) &
                                        (city_weather_df['wind_speed'] < 10)
                                     ]
ideal_cities_df = ideal_cities_df[['city', 'latitude', 'longitude', 'temperature', 'humidity', 'cloudiness', 'wind_speed']]
ideal_cities_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
params = {
    'radius': 10000,
    'types': "lodging",
    'key': gkey
}

hotel = []

for index, row in ideal_cities_df.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    params['location'] = f"{lat},{lon}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    try:
        response = requests.get(base_url, params=params).json()
        hotel.append(response["results"][0]["name"])
        print(f"{index} processed successfully")
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [ ]:
hotel

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
